In [16]:
import os, sys
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
path = os.getcwd() + '/data/'
from sklearn.preprocessing import LabelEncoder


In [2]:
# %%bash
# conda create -n ipykernel_py2 python=2 ipykernel
# source activate ipykernel_py2    # On Windows, remove the word 'source'
# python -m ipykernel install --user

In [3]:
# to solve kernel dying problem
# %%bash
# conda install nomkl

In [4]:

log1 = pd.read_csv(path + 'student_log_{}.csv'.format(1))
initial_col_order = [col.lower() for col in list(log1.columns)]

data = pd.DataFrame()
for i in range(1, 10):
    student_log = pd.read_csv(path + 'student_log_{}.csv'.format(i))
    student_log.columns = [col.lower() for col in student_log]
    data = pd.concat([data, student_log], ignore_index=True)



/usr/local/anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data = data[initial_col_order]
data.shape
data.head()

training = pd.read_csv(path + 'training_label.csv').drop(['AveCorrect'], axis=1)
training.columns = [col.lower() for col in training.columns]
training.shape

# testing on a little dataset
df = data
del data
df1 = df.merge(training, on='itest_id', how="left")

# if memory error
del df

# actions = df1[df1.isstem.notnull()]
actions = pd.DataFrame(df1[df1.isstem.notnull()])

print("training set size: ", training.shape[0], "actions for # of students: ", len(actions.itest_id.unique()))
print("sample size useless for training: ", training.shape[0] - len(actions.itest_id.unique()))


training set size:  514 actions for # of students:  449
sample size useless for training:  65


In [6]:
# merged training data set
stem = actions[actions.isstem == 1]
nonstem = actions[actions.isstem == 0]

stem.shape, nonstem.shape

((77088, 79), (187471, 79))

In [7]:
# prepare data
# actions_clear = actions.drop(['actionid', 'problemid', 'assignmentid', 'assistmentid', 'starttime', 'endtime'], axis=1)
actions_clear = actions.drop(['actionid', 'problemid', 'assignmentid', 'assistmentid'], axis=1)
actions_clear.head()

,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln,schoolid,mcas,isstem
1056,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.262675,0.060808,0.000000,0.889219,0.005797,0.271,0.0778992,2.0,34.0,0.0
1057,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.696486,0.000000,0.009561,0.108417,0.001483,0.0778992,0.225856,2.0,34.0,0.0
1058,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.225856,0.483008,2.0,34.0,0.0
1059,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.483008,0.74529,2.0,34.0,0.0
1060,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.898073,0.000000,0.009561,0.468252,0.001483,0.74529,0.900994,2.0,34.0,0.0


In [29]:
print(actions_clear['skill'].nunique())
new_actions = actions_clear

93


In [30]:
# dummy columns
# nominal_columns = ['sy assistments usage','skill']
# dummy_df = pd.get_dummies(actions_clear['problemtype'])
# print(dummy_df.columns)
# print('0' in dummy_df)
# dummy_df = dummy_df.drop([u'0', u'1'], axis=1)
# print('0' in dummy_df)

# new_actions = pd.concat([actions_clear, dummy_df], axis=1)
# print('0' in new_actions)

# nominal_columns = ['sy assistments usage', 'skill']
# dummy_df = pd.get_dummies(actions_clear[nominal_columns])
# new_actions = pd.concat([new_actions, dummy_df], axis=1)
# print('0' in new_actions)
# new_actions = new_actions.drop(['sy assistments usage', 'skill', 'problemtype'], axis=1)

In [31]:
target = new_actions['isstem']
new_actions = new_actions.drop(['isstem'], axis=1)


In [32]:
new_actions['ln'] = new_actions['ln'].astype(float)
new_actions['ln-1'] = new_actions['ln-1'].astype(float)


In [33]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)


height has been deprecated.



In [34]:
new_actions = new_actions.drop(['sumtime3sdwhen3rowright'], axis=1)

### Add here part to run one simple decision tree and visualize it

In [35]:
# print(new_actions.isnull().sum())
# print(new_actions.shape)
# print(target.shape)
# categorical - "sy assistments usage", "skill", "problemtype"

In [36]:
# convert labels (categorical)
le = LabelEncoder()
new_actions["sy assistments usage"] = le.fit_transform(new_actions["sy assistments usage"])
sy_labels = le.classes_
new_actions["skill"] = le.fit_transform(new_actions["skill"])
skill_labels = le.classes_
new_actions["problemtype"] = le.fit_transform(new_actions["problemtype"])
problemtype_labels = le.classes_
# le.fit_transform(new_actions["sy assistments usage"])

In [37]:
clf = DecisionTreeClassifier(random_state=0)
clf = clf.fit(new_actions,target)

In [63]:
# %%bash
# conda install -c anaconda graphviz
# conda install -c conda-forge pydotplus
# conda install -c conda-forge python-graphviz

In [64]:
import graphviz
from sklearn.tree import export_graphviz
import io
import pydotplus
import matplotlib as plt
path = os.getcwd() + "/graph/"

In [65]:
# define function to plot the tree
def show_tree(tree, path):
#     f = io.BytesIO()
    f = io.StringIO()
    export_graphviz(tree,
                    feature_names=new_actions.columns,
                    filled=True,
                    rounded=True, out_file=f)
#     pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png('dtree'+'.png')

In [66]:
show_tree(clf, path)
# dot_data = StringIO()

# export_graphviz(tree, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True)

# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

In [ ]:
# f = io.BytesIO()
# # tree.export_graphviz(tree_in_forest, out_file=f)
# export_graphviz(clf,
#             feature_names=new_actions.columns,
#             filled=True,
#             rounded=True, out_file=f)
# # os.system('dot -Tpng tree.dot -o tree.png')
# pydotplus.graph_from_dot_data(f.getvalue()).write_png('dtree'+ str(i_tree) +'.png')
# i_tree = i_tree + 1

### Ensemble - Random Forest

In [14]:
import sklearn.ensemble
from sklearn.model_selection import cross_val_score

In [14]:
import sklearn.ensemble

nfeatures = int(round(np.sqrt(len(new_actions.columns))))
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_features=nfeatures)

clf.fit(new_actions, target)
clf.feature_importances_

array([  7.39988660e-02,   8.39276662e-02,   8.76456523e-02,
         7.40817322e-02,   7.81335630e-02,   7.09634317e-02,
         6.67819167e-02,   7.13745846e-02,   7.00823239e-02,
         7.60903877e-02,   7.44789523e-02,   1.30754251e-03,
         1.71533474e-04,   2.52172190e-04,   2.55295141e-04,
         9.35272344e-04,   1.82742358e-03,   1.70005378e-04,
         1.97375684e-05,   5.60266293e-04,   1.09989070e-04,
         9.57749194e-04,   1.28732070e-03,   2.56394513e-05,
         2.67682100e-04,   2.02895744e-03,   1.97481720e-03,
         6.76730371e-04,   9.98921193e-04,   3.07015968e-03,
         1.11989884e-03,   9.09837524e-04,   7.22567006e-03,
         2.02587359e-03,   3.40444716e-04,   0.00000000e+00,
         3.44535782e-04,   3.15349625e-05,   1.93906780e-03,
         1.35723408e-03,   1.36975412e-03,   3.17885302e-04,
         1.06651801e-04,   7.13036870e-03,   3.86016183e-05,
         4.73138317e-04,   2.14075408e-04,   2.91068322e-03,
         2.73319426e-03,

In [15]:
weights = list(clf.feature_importances_)
names = new_actions.columns

features= dict(zip(names, weights))
features

sorted(features.items(), key=lambda x: x[1])


[('skill_slope', 0.0),
 ('responseischosen', 0.0),
 ('skill_similar-triangles', 2.071256108507909e-07),
 ('skill_multiplying-positive-negative-numbers', 7.8734897503289098e-07),
 ('skill_adding-decimals', 9.2106707193616203e-07),
 ('skill_integers', 2.6712940112021611e-06),
 ('skill_g-geometry', 3.0391006703415461e-06),
 ('interfacepopupmenuquestion1', 3.0652297955283932e-06),
 ('skill_reduce-fraction', 3.6692297176860159e-06),
 ('skill_graph interpretation', 3.9387541576143123e-06),
 ('skill_m-measurement', 4.0510838832981456e-06),
 ('skill_simple-calculation', 4.388276555155415e-06),
 ('skill_area-concept', 5.1639316426864601e-06),
 ('skill_stem-and-leaf-plot', 5.370718969758234e-06),
 ('skill_circumference', 6.4703147961894927e-06),
 ('skill_surface-area-and-volume', 6.9790705678349435e-06),
 ('skill_of-means-multiply', 8.7727151816572855e-06),
 ('skill_meaning-of-pi', 8.7738268114490813e-06),
 ('skill_mode', 9.5621772433412868e-06),
 ('skill_area-of-circle', 9.7817927214005071e-06)

In [16]:
# train on the most important features
from sklearn.model_selection import cross_val_score

most_important_features = ['schoolid', 'mcas', 'averesengcon', 'averesfrust', 'averesbored', 'averesconf', 'itest_id', 'avecorrect', 'averesgaming', 'averesofftask', 'numactions', 'aveknow', 'avecarelessness']
important_data = new_actions[most_important_features]

clf2 = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_features=nfeatures)
cross_val_score(clf2, important_data, target)

array([ 0.40811004,  0.64316331,  0.59008233])

In [18]:
np.mean([ 0.40811004,  0.64316331,  0.59008233])

0.54711856000000003

In [15]:
# Standardizing the features
from sklearn.preprocessing import StandardScaler
norm_data = StandardScaler().fit_transform(new_actions)

In [16]:
# run PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=15)

principal_components = pca.fit_transform(norm_data)
pca.components_

array([[-0.00999417,  0.16724447,  0.15362711, ...,  0.00631832,
         0.01961439,  0.00250546],
       [-0.00663063, -0.00105228, -0.00372382, ..., -0.01645343,
         0.00768322, -0.02376904],
       [-0.00967382, -0.19507752, -0.19539393, ..., -0.0025047 ,
         0.02612087,  0.02187113],
       ..., 
       [-0.00182556,  0.00453122,  0.00122058, ...,  0.00859978,
         0.01163515,  0.00511348],
       [-0.02367464, -0.04783888, -0.03107252, ..., -0.02809883,
        -0.0233675 ,  0.09515509],
       [-0.08351236,  0.14045855,  0.11645575, ...,  0.00972406,
         0.02219522,  0.09084398]])

In [27]:
pca = PCA(.95)
principal_components2 = pca.fit_transform(norm_data)
pca.n_components_
principal_df = pd.DataFrame(data = principal_components2)

In [28]:
principal_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131
0,4.721957,-2.990386,4.175463,3.979769,2.909892,2.114322,-2.017010,2.941096,-0.633159,0.666211,-1.281523,-0.233071,-0.006331,0.094175,1.860821,-0.025482,-1.019970,1.110827,0.173853,0.388785,-1.670511,0.888727,0.437602,-0.188692,-1.979957,1.260014,1.609392,2.431231,0.691084,0.489710,-2.023069,-3.483703,4.872012,-5.309638,-0.612123,-2.398216,-1.783121,0.676569,-1.526831,2.336184,-1.969128,1.910754,-1.073014,-0.986451,0.723105,0.009984,-0.757361,0.611578,0.253614,0.043974,0.497858,-0.008133,-0.055043,0.078109,0.070610,-0.150342,0.604334,-0.224045,0.197873,-0.086374,0.395519,-0.211742,-0.325839,-0.487790,0.509160,0.584289,-0.047662,0.234658,-0.386741,0.295819,-0.166622,0.482932,-0.324575,0.273718,0.333857,-0.049886,-0.387976,0.292231,0.398636,0.059446,-0.051024,-0.026246,0.007688,0.099168,-0.149828,0.018666,-0.319712,-0.089608,0.160970,0.048067,0.166903,0.202219,-0.101787,0.019377,-0.190113,0.193224,-0.365210,-0.089388,0.776540,-0.759781,-0.838304,-1.215003,1.113724,-0.050333,-0.285653,2.356537,-1.650987,-5.302473,6.467669,-2.250715,0.727720,0.544780,1.082552,-0.877306,1.282988,-2.439585,0.303839,-0.229491,1.967992,0.661285,-0.899639,1.383522,1.443620,1.332492,-1.131473,-0.307523,-0.041564,0.156636,1.742613,1.062647,-1.660185,2.300211
1,0.931975,-1.467448,-0.953566,-2.302399,0.659696,-1.136841,-0.880906,3.186752,-0.322791,-0.097296,-0.667503,-0.626014,-0.353842,0.826150,1.867795,-0.457168,-0.261641,0.262353,0.239005,0.458069,-1.596028,1.752592,0.181851,-0.094255,-2.022593,0.849027,1.391607,1.635637,-0.058052,1.025159,-1.813996,-2.973770,4.159165,-4.460402,-1.130818,-1.611053,-1.889145,0.573507,-1.069537,2.026132,-1.345137,1.577318,-0.738784,-1.133639,0.708475,0.150490,-0.784755,0.612868,0.489545,0.077228,0.481420,0.009882,0.140415,0.051710,0.203714,-0.154456,0.572731,-0.264592,0.144053,-0.009986,0.405798,-0.251346,-0.168627,-0.337340,0.440541,0.420339,-0.097136,0.156494,-0.332271,0.224967,-0.214546,0.434682,-0.137754,0.180231,0.353574,-0.060089,-0.324471,0.163632,0.445845,-0.010562,0.035773,-0.064327,0.019088,0.067367,-0.189703,-0.051359,-0.317825,-0.019780,0.138454,0.052778,0.129097,0.190959,-0.086410,-0.015024,-0.100852,0.098150,-0.237085,0.022615,0.878213,-0.677355,-0.511655,-0.932105,1.093315,-0.120158,-0.462597,1.863820,-0.975903,-4.401803,5.663062,-1.740280,1.631942,0.571260,0.295643,-1.042785,2.308389,-1.013656,-0.373545,0.543021,1.407702,1.271174,0.066622,1.006387,1.730982,0.699187,-0.528158,0.057739,0.017640,1.073746,0.563891,1.417233,-1.810868,1.706050
2,0.690700,-1.120766,-1.315542,-2.207134,1.185317,-0.359981,-0.753605,2.191082,-0.304806,0.346604,-0.835272,-0.391650,-0.527081,1.293923,1.206739,-0.456139,-0.290715,0.668032,0.135408,0.685812,-1.589404,2.098666,0.020772,0.301943,-1.507694,0.702007,1.291190,1.365626,-0.001136,0.929349,-1.430076,-2.318872,3.214442,-3.575740,-0.630643,-1.110551,-1.557436,0.884548,-0.594107,1.590624,-0.949581,1.198751,-0.713607,-1.225617,0.589683,0.131734,-0.856613,0.488464,0.400520,0.105394,0.378276,-0.035815,0.084798,0.045315,0.125175,-0.140559,0.495351,-0.258997,0.129637,-0.019612,0.293739,-0.209571,-0.207668,-0.296891,0.358273,0.269808,-0.050019,0.095398,-0.279172,0.153788,-0.159003,0.322295,-0.104754,0.152365,0.267993,-0.061241,-0.253260,0.150402,0.390688,0.019390,0.089152,-0.081506,-0.038562,0.024821,-0.205482,-0.052410,-0.256438,0.017286,0.152850,0.063293,0.109318,0.194906,-0.065726,-0.050568,-0.013762,0.064373,-0.212653,0.011353,0.872244,-0.495494,-0.365042,-0.840880,1.028650,-0.205886,-0.338845,1.572553,-0.564235,-3.625362,3.994318,-1.475592,1.218810,0.360027,0.095171,-0.7

In [29]:
# random forest on the new reduced dimensions
clf3 = sklearn.ensemble.RandomForestClassifier(n_estimators=200)
cross_val_score(clf3, principal_df, target)

array([ 0.69941148,  0.67400721,  0.69636904])

In [ ]:
pca.explained_variance_

In [30]:
np.mean([ 0.70208761,  0.66749824,  0.66377883]) # 0.677788
np.mean([ 0.69941148,  0.67400721,  0.69636904]) # 0.6899

0.68992924333333328